# 模块七：AI 应用实验手册

本笔记本创建了一个完整的环境，用于运行使用 **Microsoft Florence-2-base** 模型的 AI 视觉演示。

## 步骤 0：安装依赖
我们需要安装所需的库，包括 `transformers`、`fastapi`、`uvicorn` 以及用于调用云端 API 的 `openai`。

In [ ]:
# 安装依赖 (兼容 Mac/Linux/Windows)
# 使用 transformers==4.51.3 以确保稳定性与速度
%pip install transformers==4.51.3 timm einops pillow "qrcode[pil]" fastapi uvicorn openai -i https://pypi.tuna.tsinghua.edu.cn/simple

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


zsh:1: no matches found: qrcode[pil]
Note: you may need to restart the kernel to use updated packages.


## Lesson 25: 3. 你的第一个视觉脚本:看图说话

我们将加载 **Microsoft Florence-2-base** 模型。这是一个 VLM（视觉语言模型），能够生成详细的图像字幕。

**Mac 用户注意：** 我们启用了 MPS 回退机制，并使用 `mock` 技巧来绕过 `flash_attn` 要求。

In [ ]:
import sys
import os

# 1. 启用 MPS 回退 (必须在导入 torch 前设置!)
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

from unittest.mock import MagicMock
import torch
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image

# 2. Mock flash_attn 检查以兼容 Mac
sys.modules["flash_attn"] = MagicMock()
sys.modules["flash_attn"].__spec__ = MagicMock()

# 3. 加载模型
# 优先检查本地模型 ./models/florence-2-base
local_model_path = "./models/florence-2-base"
if os.path.exists(local_model_path):
    model_id = local_model_path
    print(f"📂 使用本地模型: {model_id}")
else:
    model_id = "microsoft/Florence-2-base"
    print(f"☁️ 使用远程模型: {model_id}")

# 自动检测设备 (Mac 用 MPS, Nvidia 用 CUDA, 否则用 CPU)
if torch.backends.mps.is_available():
    device = "mps"
    torch_dtype = torch.float16
elif torch.cuda.is_available():
    device = "cuda"
    torch_dtype = torch.float16
else:
    device = "cpu"
    torch_dtype = torch.float32

print(f"使用设备: {device.upper()}")
print("正在加载模型... (可能需要一分钟)")

try:
    model = AutoModelForCausalLM.from_pretrained(
        model_id, 
        trust_remote_code=True, 
        torch_dtype=torch_dtype,
        local_files_only=True
    ).to(device)
    
    processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
    print("✅ 模型加载成功!")
except Exception as e:
    print(f"❌ 模型加载失败: {e}")

## Lesson 25: 4. 代码解析 (Code Analysis: Inference Test)
让我们用一个简单的函数来测试模型。您可以在这里上传图像路径或 URL。

In [ ]:
def analyze_image(image_path, task="<MORE_DETAILED_CAPTION>"):
    if image_path.startswith("http"):
        import requests
        image = Image.open(requests.get(image_path, stream=True).raw)
    else:
        image = Image.open(image_path)
    
    if image.mode != "RGB":
        image = image.convert("RGB")

    inputs = processor(text=task, images=image, return_tensors="pt").to(device, torch_dtype)

    # 第一次运行可能会慢 (Warmup)，因为 MPS 需要编译图
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        do_sample=False,
        num_beams=3,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(generated_text, task=task, image_size=(image.width, image.height))
    
    return parsed_answer[task]

# model_warmup() # 可选: 预热一次

# 测试 (如果不存在则创建一个小的测试图片)
dummy_img_path = "test_image.jpg"
if not os.path.exists(dummy_img_path):
    Image.new('RGB', (100, 100), color = 'red').save(dummy_img_path)
    
print("正在分析图片...")
try:
    print(analyze_image(dummy_img_path))
except Exception as e:
    print(f"错误: {e}")

## Lesson 25: 9. 最终实验: 你的第一个 AI App
恭喜！您已经到了最后一步。我们将生成前端代码 (`index.html`) 和后端代码 (`app.py`)，然后启动服务。

### 1. 生成前端 (index.html)

In [ ]:
%%writefile index.html
<!DOCTYPE html>
<html lang="zh-CN">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">
    <title>AI Vision Demo (Florence-2)</title>
    <style>
        :root {
            --primary-color: #4CAF50;
            --bg-color: #1e1e1e;
            --card-bg: #252526;
            --text-color: #ddd;
        }

        body {
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Open Sans', 'Helvetica Neue', sans-serif;
            background-color: var(--bg-color);
            color: var(--text-color);
            display: flex;
            flex-direction: column;
            align-items: center;
            min-height: 100vh;
            margin: 0;
            padding: 20px;
        }

        .container {
            width: 100%;
            max-width: 600px;
            display: flex;
            flex-direction: column;
            gap: 20px;
        }

        h1 {
            text-align: center;
            color: var(--primary-color);
            margin-bottom: 5px;
        }

        .subtitle {
            text-align: center;
            color: #888;
            font-size: 0.9em;
            margin-bottom: 20px;
        }

        .card {
            background-color: var(--card-bg);
            padding: 20px;
            border-radius: 12px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.3);
        }

        /* QR Code Section - Hidden on Mobile */
        .qr-section {
            text-align: center;
            display: flex;
            flex-direction: column;
            align-items: center;
            gap: 10px;
        }

        .qr-section img {
            width: 180px;
            height: 180px;
            border-radius: 8px;
            border: 4px solid white;
        }

        @media (max-width: 600px) {
            .qr-section {
                display: none;
            }

            .mobile-hidden {
                display: none;
            }
        }

        @media (min-width: 601px) {
            .desktop-hidden {
                display: none;
            }
        }

        /* Upload Section */
        .upload-area {
            border: 2px dashed #444;
            border-radius: 8px;
            padding: 30px;
            text-align: center;
            cursor: pointer;
            transition: border-color 0.3s;
            position: relative;
        }

        .upload-area:hover {
            border-color: var(--primary-color);
        }

        .upload-area input[type="file"] {
            position: absolute;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            opacity: 0;
            cursor: pointer;
        }

        .icon {
            font-size: 48px;
            margin-bottom: 10px;
            display: block;
        }

        .btn {
            background-color: var(--primary-color);
            color: white;
            border: none;
            padding: 12px 24px;
            border-radius: 6px;
            font-size: 16px;
            font-weight: bold;
            cursor: pointer;
            width: 100%;
            margin-top: 15px;
            transition: background-color 0.2s;
        }

        .btn:disabled {
            background-color: #555;
            cursor: not-allowed;
        }

        /* Result Section */
        .preview-img {
            max-width: 100%;
            border-radius: 8px;
            margin-bottom: 15px;
            border: 1px solid #444;
        }

        .result-text {
            font-size: 1.1em;
            line-height: 1.6;
            background: #333;
            padding: 15px;
            border-radius: 8px;
            border-left: 4px solid var(--primary-color);
        }

        .loader {
            display: inline-block;
            width: 20px;
            height: 20px;
            border: 3px solid rgba(255, 255, 255, .3);
            border-radius: 50%;
            border-top-color: #fff;
            animation: spin 1s ease-in-out infinite;
            margin-right: 10px;
            vertical-align: middle;
        }

        @keyframes spin {
            to {
                transform: rotate(360deg);
            }
        }

        .hidden {
            display: none;
        }
    </style>
</head>

<body>

    <div class="container">
        <div>
            <h1>👁️ AI Vision Demo</h1>
            <div class="subtitle">Powered by Florence-2 VLM (v1.0)</div>
        </div>

        <!-- Desktop Only: QR Code -->
        <div class="card qr-section">
            <img src="/qrcode" alt="Scan to use on mobile">
            <p>📱 手机扫码体验</p>
        </div>

        <!-- Main Interaction Area -->
        <div class="card">
            <!-- File Upload -->
            <div class="upload-area" id="drop-zone">
                <span class="icon">📷</span>
                <p class="mobile-hidden">点击上传图片</p>
                <p class="desktop-hidden">点击拍照或上传图片</p>
                <input type="file" id="file-input" accept="image/*">
            </div>

            <!-- Webcam Trigger -->
            <button id="cam-btn" class="btn mobile-hidden" style="background-color: #333; margin-top: 10px;">💻
                使用电脑摄像头</button>

            <!-- Camera View (Hidden by default) -->
            <div id="camera-container" class="hidden" style="margin-top: 20px; text-align: center;">
                <video id="video-feed" autoplay playsinline
                    style="width: 100%; border-radius: 8px; border: 2px solid #4CAF50;"></video>
                <button id="snap-btn" class="btn">📸 拍照</button>
                <button id="close-cam-btn" class="btn" style="background-color: #666;">取消</button>
            </div>

            <div id="preview-container" class="hidden" style="margin-top: 20px;">
                <canvas id="canvas" class="hidden"></canvas>
                <img id="preview-img" class="preview-img" src="" alt="Preview">
                <button id="upload-btn" class="btn">🚀 开始识别</button>
            </div>
        </div>

        <!-- Result Area -->
        <div id="result-card" class="card hidden">
            <h3>📝 识别结果</h3>
            <div id="result-content" class="result-text"></div>
        </div>
    </div>

    <script>
        // Global Audio Helpers
        const fileInput = document.getElementById('file-input');
        const previewContainer = document.getElementById('preview-container');
        const previewImg = document.getElementById('preview-img');
        const uploadBtn = document.getElementById('upload-btn');
        const resultCard = document.getElementById('result-card');
        const resultContent = document.getElementById('result-content');

        // Camera Elements
        const camBtn = document.getElementById('cam-btn');
        const cameraContainer = document.getElementById('camera-container');
        const video = document.getElementById('video-feed');
        const snapBtn = document.getElementById('snap-btn');
        const closeCamBtn = document.getElementById('close-cam-btn');
        const canvas = document.getElementById('canvas');
        let stream = null;
        let blobToSend = null; // Store blob for upload

        // 1. File Selection
        fileInput.addEventListener('change', function (e) {
            if (this.files && this.files[0]) {
                const file = this.files[0];
                blobToSend = file; // Set blob
                const reader = new FileReader();
                reader.onload = function (e) {
                    previewImg.src = e.target.result;
                    previewContainer.classList.remove('hidden');
                    cameraContainer.classList.add('hidden');
                    resultCard.classList.add('hidden');
                    stopCamera();
                }
                reader.readAsDataURL(file);
            }
        });

        // 2. Open Camera
        camBtn.addEventListener('click', async () => {
            try {
                stream = await navigator.mediaDevices.getUserMedia({ video: { facingMode: "environment" } });
                video.srcObject = stream;
                cameraContainer.classList.remove('hidden');
                previewContainer.classList.add('hidden');
                resultCard.classList.add('hidden');
            } catch (err) {
                alert("无法访问摄像头: " + err.message);
            }
        });

        // 3. Close Camera
        closeCamBtn.addEventListener('click', () => {
            stopCamera();
            cameraContainer.classList.add('hidden');
        });

        function stopCamera() {
            if (stream) {
                stream.getTracks().forEach(track => track.stop());
                stream = null;
            }
        }

        // 4. Snap Photo
        snapBtn.addEventListener('click', () => {
            const context = canvas.getContext('2d');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            context.drawImage(video, 0, 0, canvas.width, canvas.height);

            // Convert to Blob
            canvas.toBlob(blob => {
                blobToSend = blob;
                previewImg.src = URL.createObjectURL(blob);
                previewContainer.classList.remove('hidden');
                cameraContainer.classList.add('hidden');
                stopCamera();
            }, 'image/jpeg');
        });

        // 5. Upload Logic
        // 5. Upload Logic
        // 5. Upload Logic (Refactored for Lesson 26 Pipeline: Vision -> Translation)
        uploadBtn.addEventListener('click', async function () {
            if (!blobToSend) return alert("请先选择图片或拍照");

            const formData = new FormData();
            formData.append('file', blobToSend, "capture.jpg");

            // UI State: Loading Vision
            uploadBtn.disabled = true;
            let startTime = Date.now();
            let timerInterval = setInterval(() => {
                let elapsed = ((Date.now() - startTime) / 1000).toFixed(1);
                uploadBtn.innerHTML = `<span class="loader"></span> 正在推理中... (${elapsed}s)`;
            }, 100);

            resultContent.innerText = '';

            try {
                // ==========================
                // Step 1: Vision (Florence-2)
                // ==========================
                const response = await fetch('/upload', { method: 'POST', body: formData });
                if (!response.ok) throw new Error('Network error on /upload');

                const data = await response.json();

                // Clear Vision timer
                clearInterval(timerInterval);

                if (data.error) {
                    resultContent.innerText = "❌ 错误: " + data.error;
                    return;
                }

                const visionTime = data.cost_time;
                const captionEn = data.caption_en || data.label; // Fallback for old API

                // Display Step 1 Result (Immediate English)
                resultCard.classList.remove('hidden');
                resultContent.innerHTML = `
                    <div style="margin-bottom:10px;">
                        <strong>🇬🇧 EN:</strong> ${captionEn}
                        <div style="font-size:0.8em; color:#888;">⏱️ 视觉耗时: ${visionTime}s</div>
                    </div>
                `;

                } catch (error) {
                clearInterval(timerInterval);
                alert('上传失败，请重试');
                console.error(error);
                resultContent.innerText = "❌ 系统错误: " + error.message;
            } finally {
                // UI State: Reset
                uploadBtn.disabled = false;
                uploadBtn.innerText = '🚀 开始识别';
            }
        });
    </script>
</body>

</html>



### 2. 生成后端 (app.py)

In [ ]:
%%writefile app.py
import os
# Fix for MPS 'aten::isin' error: Enable fallback to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
# Suppress huggingface/tokenizers parallelism warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import uvicorn
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import HTMLResponse, Response
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import io
import socket
import qrcode
import sys
import torch
import pathlib
import pathlib
import time
import warnings # Added for warning suppression

# Filter out specific library warnings to keep console clean for students
warnings.filterwarnings("ignore", message=".*To copy construct from a tensor.*")
warnings.filterwarnings("ignore", category=UserWarning, module="transformers")

# 1. 初始化 FastAPI 应用
app = FastAPI(title="Lesson 25 Vision Demo (Florence-2)")

# Mock flash_attn for Mac compatibility
from unittest.mock import MagicMock
sys.modules["flash_attn"] = MagicMock()
sys.modules["flash_attn"].__spec__ = MagicMock()

# Helper: Get Local IP
def get_local_ip():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip = s.getsockname()[0]
        s.close()
        return ip
    except:
        return "127.0.0.1"

# 2. 预加载模型 (使用 Microsoft Florence-2-base)
# 优先检查本地模型
local_model_path = "./models/florence-2-base"
if os.path.exists(local_model_path):
    print(f"📂 发现本地模型 path: {local_model_path}")
    model_id = local_model_path
else:
    print(f"⚠️ 未找到本地模型，准备从 HuggingFace 下载...")
    model_id = "microsoft/Florence-2-base"
# 自动检测设备: 优先 MPS (Mac), 其次 CPU
device = "mps" if torch.backends.mps.is_available() else "cpu"
# Florence-2 在 GPU/MPS 上通常使用 float16，但在 CPU 上使用 float32
torch_dtype = torch.float16 if device != "cpu" else torch.float32

model = None
processor = None

try:
    print("="*50)
    print(f"🚀 正在加载 Florence-2 模型...")
    print(f"📂 模型来源: {model_id}")
    print(f"🖥️  运行设备: {device.upper()}")
    
    # 如果发现是本地路径，强制离线模式
    local_files_only = False
    if os.path.isdir(model_id):
        print(f"🔌 检测到本地路径，启用离线模式 (local_files_only=True)")
        local_files_only = True
        os.environ["HF_DATASETS_OFFLINE"] = "1"
        os.environ["TRANSFORMERS_OFFLINE"] = "1"
    
    if model_id == "microsoft/Florence-2-base" and not local_files_only:
        print("第一次运行需要下载模型 (约 1.5GB)，请耐心等待...")
    
    model = AutoModelForCausalLM.from_pretrained(
        model_id, 
        trust_remote_code=True, 
        torch_dtype=torch_dtype,
        local_files_only=local_files_only
    ).to(device)
    
    processor = AutoProcessor.from_pretrained(
        model_id, 
        trust_remote_code=True,
        local_files_only=local_files_only
    )
    
    print("✨ Florence-2 模型加载成功！")
    print("="*50)
except Exception as e:
    import traceback
    print(f"\n❌ 模型加载失败: {e}")
    print("可能是 transformer 版本过低或依赖缺失。请尝试运行: pip install -U transformers timm einops")
    traceback.print_exc()

# 3. 模型预热 (Warmup) - 消除第一次运行的卡顿
def warmup_model():
    if not model or not processor: return
    print("🔥 正在预热模型... (消除首次推理卡顿)")
    try:
        # 创建一个极其微小的 Dummy 输入
        dummy_img = Image.new('RGB', (64, 64), color='white')
        dummy_prompt = "<CAPTION>"
        inputs = processor(text=dummy_prompt, images=dummy_img, return_tensors="pt").to(device, torch_dtype)
        # 强制运行一次生成
        model.generate(
            input_ids=inputs["input_ids"],
            pixel_values=inputs["pixel_values"],
            max_new_tokens=5, 
            do_sample=False,
            num_beams=1,
        )
        print("✅ 模型预热完成！")
    except Exception as e:
        print(f"⚠️ 预热失败 (不影响主功能): {e}")

# 执行预热
warmup_model()

@app.get("/", response_class=HTMLResponse)
def home():
    """Serve the frontend HTML"""
    return pathlib.Path("index.html").read_text(encoding="utf-8")

@app.get("/qrcode")
def get_qr_image():
    """Generate QR Code for the server URL"""
    ip = get_local_ip()
    url = f"http://{ip}:8000"
    print(f"📱 QR Code URL: {url}")
    
    qr = qrcode.QRCode(box_size=10, border=4)
    qr.add_data(url)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")
    
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    buf.seek(0)
    return Response(content=buf.getvalue(), media_type="image/png")

@app.post("/upload")
async def upload_image(file: UploadFile = File(...)):
    """
    接收手机上传的图片，并返回详细描述 (Detailed Caption)
    """
    global model, processor
    
    if not model or not processor:
        return {"error": "Model not loaded. Check server logs."}

    try:
        # 读取图片数据
        contents = await file.read()
        image = Image.open(io.BytesIO(contents))
        if image.mode != "RGB":
            image = image.convert("RGB")
        
        # 构造提示词 task
        prompt = "<MORE_DETAILED_CAPTION>"
        
        # 打印日志到控制台
        print(f"\n📸 收到图片: {file.filename}")
        
        # 计时开始
        start_time = time.time()
        
        # 预处理输入
        inputs = processor(text=prompt, images=image, return_tensors="pt").to(device, torch_dtype)

        # 生成描述
        generated_ids = model.generate(
            input_ids=inputs["input_ids"],
            pixel_values=inputs["pixel_values"],
            max_new_tokens=1024,
            do_sample=False,
            num_beams=3,
        )
        
        # 解码输出
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
        
        # 后处理 (Florence-2 特有)
        parsed_answer = processor.post_process_generation(
            generated_text, 
            task=prompt, 
            image_size=(image.width, image.height)
        )
        
        # 计时结束
        end_time = time.time()
        cost_time = round(end_time - start_time, 2)
        
        # 获取最终文本
        caption = parsed_answer[prompt]
        
        # 打印日志到控制台
        print(f"⏱️ 推理耗时: {cost_time}s")
        print(f"🤖 Florence-2 描述:\n{caption}")
        
        return {
            "label": caption,
            "cost_time": cost_time
        }
    except Exception as e:
        import traceback
        print(f"❌ 处理图片时发生错误: {e}")
        traceback.print_exc()
        return {"error": str(e)}

def get_local_ip():
    """获取本机局域网IP"""
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip = s.getsockname()[0]
        s.close()
        return ip
    except:
        return "127.0.0.1"

def print_qr_code(url):
    """在终端打印二维码"""
    try:
        qr = qrcode.QRCode()
        qr.add_data(url)
        qr.print_ascii(invert=True)
        print(f"\n📱 手机扫码体验: {url}")
    except Exception:
        print(f"\n⚠️ 无法生成二维码，请手动访问: {url}")

if __name__ == "__main__":
    ip = get_local_ip()
    port = 8000
    url = f"http://{ip}:{port}/docs"
    
    print("\n" + "="*50)
    print(f"🚀 服务启动中...")
    print(f"Running on: {url}")
    print("="*50)
    
    # 打印二维码供扫描
    print_qr_code(url)
    
    print("\n按 Ctrl+C 停止服务")
    
    uvicorn.run(app, host="0.0.0.0", port=port)


### 3. 运行服务
最后，在终端运行以下命令

In [ ]:
!python app.py

## Lesson 26: 8. 再次迭代: 引入 云端翻译 & 语音合成 (TTS) 
在这个版本中，我们将引入:
- **SiliconFlow API**: 用于 DeepSeek 翻译 (英 -> 中) 和 CosyVoice2 语音合成。
- **Florence-2**: 依然负责视觉识别。

我们需要创建一个新的前端 `lesson26.html` 和新的后端 `app_v2.py`。


In [ ]:
%%writefile lesson26.html
<!DOCTYPE html>
<html lang="zh-CN">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">
    <title>AI Vision Demo (Florence-2)</title>
    <style>
        :root {
            --primary-color: #4CAF50;
            --bg-color: #1e1e1e;
            --card-bg: #252526;
            --text-color: #ddd;
        }

        body {
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Open Sans', 'Helvetica Neue', sans-serif;
            background-color: var(--bg-color);
            color: var(--text-color);
            display: flex;
            flex-direction: column;
            align-items: center;
            min-height: 100vh;
            margin: 0;
            padding: 20px;
        }

        .container {
            width: 100%;
            max-width: 600px;
            display: flex;
            flex-direction: column;
            gap: 20px;
        }

        h1 {
            text-align: center;
            color: var(--primary-color);
            margin-bottom: 5px;
        }

        .subtitle {
            text-align: center;
            color: #888;
            font-size: 0.9em;
            margin-bottom: 20px;
        }

        .card {
            background-color: var(--card-bg);
            padding: 20px;
            border-radius: 12px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.3);
        }

        /* QR Code Section - Hidden on Mobile */
        .qr-section {
            text-align: center;
            display: flex;
            flex-direction: column;
            align-items: center;
            gap: 10px;
        }

        .qr-section img {
            width: 180px;
            height: 180px;
            border-radius: 8px;
            border: 4px solid white;
        }

        @media (max-width: 600px) {
            .qr-section {
                display: none;
            }

            .mobile-hidden {
                display: none;
            }
        }

        @media (min-width: 601px) {
            .desktop-hidden {
                display: none;
            }
        }

        /* Upload Section */
        .upload-area {
            border: 2px dashed #444;
            border-radius: 8px;
            padding: 30px;
            text-align: center;
            cursor: pointer;
            transition: border-color 0.3s;
            position: relative;
        }

        .upload-area:hover {
            border-color: var(--primary-color);
        }

        .upload-area input[type="file"] {
            position: absolute;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            opacity: 0;
            cursor: pointer;
        }

        .icon {
            font-size: 48px;
            margin-bottom: 10px;
            display: block;
        }

        .btn {
            background-color: var(--primary-color);
            color: white;
            border: none;
            padding: 12px 24px;
            border-radius: 6px;
            font-size: 16px;
            font-weight: bold;
            cursor: pointer;
            width: 100%;
            margin-top: 15px;
            transition: background-color 0.2s;
        }

        .btn:disabled {
            background-color: #555;
            cursor: not-allowed;
        }

        /* Result Section */
        .preview-img {
            max-width: 100%;
            border-radius: 8px;
            margin-bottom: 15px;
            border: 1px solid #444;
        }

        .result-text {
            font-size: 1.1em;
            line-height: 1.6;
            background: #333;
            padding: 15px;
            border-radius: 8px;
            border-left: 4px solid var(--primary-color);
        }

        .loader {
            display: inline-block;
            width: 20px;
            height: 20px;
            border: 3px solid rgba(255, 255, 255, .3);
            border-radius: 50%;
            border-top-color: #fff;
            animation: spin 1s ease-in-out infinite;
            margin-right: 10px;
            vertical-align: middle;
        }

        @keyframes spin {
            to {
                transform: rotate(360deg);
            }
        }

        .hidden {
            display: none;
        }
    </style>
</head>

<body>

    <div class="container">
        <div>
            <h1>👁️ AI Vision Demo</h1>
            <div class="subtitle">Powered by Florence-2 VLM (v1.0)</div>
        </div>

        <!-- Desktop Only: QR Code -->
        <div class="card qr-section">
            <img src="/qrcode" alt="Scan to use on mobile">
            <p>📱 手机扫码体验</p>
        </div>

        <!-- Main Interaction Area -->
        <div class="card">
            <!-- File Upload -->
            <div class="upload-area" id="drop-zone">
                <span class="icon">📷</span>
                <p class="mobile-hidden">点击上传图片</p>
                <p class="desktop-hidden">点击拍照或上传图片</p>
                <input type="file" id="file-input" accept="image/*">
            </div>

            <!-- Webcam Trigger -->
            <button id="cam-btn" class="btn mobile-hidden" style="background-color: #333; margin-top: 10px;">💻
                使用电脑摄像头</button>

            <!-- Camera View (Hidden by default) -->
            <div id="camera-container" class="hidden" style="margin-top: 20px; text-align: center;">
                <video id="video-feed" autoplay playsinline
                    style="width: 100%; border-radius: 8px; border: 2px solid #4CAF50;"></video>
                <button id="snap-btn" class="btn">📸 拍照</button>
                <button id="close-cam-btn" class="btn" style="background-color: #666;">取消</button>
            </div>

            <div id="preview-container" class="hidden" style="margin-top: 20px;">
                <canvas id="canvas" class="hidden"></canvas>
                <img id="preview-img" class="preview-img" src="" alt="Preview">
                <button id="upload-btn" class="btn">🚀 开始识别</button>
            </div>
        </div>

        <!-- Result Area -->
        <div id="result-card" class="card hidden">
            <h3>📝 识别结果</h3>
            <div id="result-content" class="result-text"></div>
        </div>
    </div>

    <script>
        // Global Audio Helpers
        window.toggleAudio = function (uniqueId) {
            const audio = document.getElementById('audio-' + uniqueId);
            const btn = document.getElementById('btn-' + uniqueId);

            if (audio.paused) {
                audio.play().catch(e => console.warn("Play failed:", e));
                btn.innerHTML = '<span>⏸️</span> 点击暂停';
                btn.style.color = '#ff9800'; // Active color
                btn.style.borderColor = '#ff9800';
            } else {
                audio.pause();
                btn.innerHTML = '<span>▶️</span> 点击播放语音';
                btn.style.color = '#4CAF50'; // Default color
                btn.style.borderColor = '#4CAF50';
            }
        };

        window.resetAudio = function (uniqueId) {
            const btn = document.getElementById('btn-' + uniqueId);
            btn.innerHTML = '<span>▶️</span> 点击播放语音';
            btn.style.color = '#4CAF50';
            btn.style.borderColor = '#4CAF50';
        };

        const fileInput = document.getElementById('file-input');
        const previewContainer = document.getElementById('preview-container');
        const previewImg = document.getElementById('preview-img');
        const uploadBtn = document.getElementById('upload-btn');
        const resultCard = document.getElementById('result-card');
        const resultContent = document.getElementById('result-content');

        // Camera Elements
        const camBtn = document.getElementById('cam-btn');
        const cameraContainer = document.getElementById('camera-container');
        const video = document.getElementById('video-feed');
        const snapBtn = document.getElementById('snap-btn');
        const closeCamBtn = document.getElementById('close-cam-btn');
        const canvas = document.getElementById('canvas');
        let stream = null;
        let blobToSend = null; // Store blob for upload

        // 1. File Selection
        fileInput.addEventListener('change', function (e) {
            if (this.files && this.files[0]) {
                const file = this.files[0];
                blobToSend = file; // Set blob
                const reader = new FileReader();
                reader.onload = function (e) {
                    previewImg.src = e.target.result;
                    previewContainer.classList.remove('hidden');
                    cameraContainer.classList.add('hidden');
                    resultCard.classList.add('hidden');
                    stopCamera();
                }
                reader.readAsDataURL(file);
            }
        });

        // 2. Open Camera
        camBtn.addEventListener('click', async () => {
            try {
                stream = await navigator.mediaDevices.getUserMedia({ video: { facingMode: "environment" } });
                video.srcObject = stream;
                cameraContainer.classList.remove('hidden');
                previewContainer.classList.add('hidden');
                resultCard.classList.add('hidden');
            } catch (err) {
                alert("无法访问摄像头: " + err.message);
            }
        });

        // 3. Close Camera
        closeCamBtn.addEventListener('click', () => {
            stopCamera();
            cameraContainer.classList.add('hidden');
        });

        function stopCamera() {
            if (stream) {
                stream.getTracks().forEach(track => track.stop());
                stream = null;
            }
        }

        // 4. Snap Photo
        snapBtn.addEventListener('click', () => {
            const context = canvas.getContext('2d');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            context.drawImage(video, 0, 0, canvas.width, canvas.height);

            // Convert to Blob
            canvas.toBlob(blob => {
                blobToSend = blob;
                previewImg.src = URL.createObjectURL(blob);
                previewContainer.classList.remove('hidden');
                cameraContainer.classList.add('hidden');
                stopCamera();
            }, 'image/jpeg');
        });

        // 5. Upload Logic
        // 5. Upload Logic
        // 5. Upload Logic (Refactored for Lesson 26 Pipeline: Vision -> Translation)
        uploadBtn.addEventListener('click', async function () {
            if (!blobToSend) return alert("请先选择图片或拍照");

            const formData = new FormData();
            formData.append('file', blobToSend, "capture.jpg");

            // UI State: Loading Vision
            uploadBtn.disabled = true;
            let startTime = Date.now();
            let timerInterval = setInterval(() => {
                let elapsed = ((Date.now() - startTime) / 1000).toFixed(1);
                uploadBtn.innerHTML = `<span class="loader"></span> 正在推理中... (${elapsed}s)`;
            }, 100);

            resultContent.innerText = '';

            try {
                // ==========================
                // Step 1: Vision (Florence-2)
                // ==========================
                const response = await fetch('/upload', { method: 'POST', body: formData });
                if (!response.ok) throw new Error('Network error on /upload');

                const data = await response.json();

                // Clear Vision timer
                clearInterval(timerInterval);

                if (data.error) {
                    resultContent.innerText = "❌ 错误: " + data.error;
                    return;
                }

                const visionTime = data.cost_time;
                const captionEn = data.caption_en || data.label; // Fallback for old API

                // Display Step 1 Result (Immediate English)
                resultCard.classList.remove('hidden');
                resultContent.innerHTML = `
                    <div style="margin-bottom:10px;">
                        <strong>🇬🇧 EN:</strong> ${captionEn}
                        <div style="font-size:0.8em; color:#888;">⏱️ 视觉耗时: ${visionTime}s</div>
                    </div>
                `;

                // ==========================
                // Step 2: Translation (Cloud Endpoint)
                // ==========================
                // Check if we need to translate (Lesson 26 feature)
                // We'll optimistically try to call /translate endpoint.

                uploadBtn.innerHTML = `<span class="loader"></span> 正在翻译中...`;

                try {
                    const transResponse = await fetch('/translate', {
                        method: 'POST',
                        headers: { 'Content-Type': 'application/json' },
                        body: JSON.stringify({ text: captionEn })
                    });

                    if (transResponse.ok) {
                        const transData = await transResponse.json();
                        const captionZh = transData.caption_zh;
                        const transTime = transData.cost_time;

                        // Append Chinese Result
                        const newHtml = `
                            <hr style="border:0; border-top:1px solid #555; margin:10px 0;">
                            <div>
                                <strong>🇨🇳 ZH:</strong> ${captionZh}
                                <div style="font-size:0.8em; color:#888;">⏱️ 翻译耗时: ${transTime}s</div>
                            </div>
                        `;
                        resultContent.innerHTML += newHtml;

                        // ==========================
                        // Step 3: TTS (Text-to-Speech)
                        // ==========================
                        uploadBtn.innerHTML = `<span class="loader"></span> 正在朗读中...`;

                        try {
                            const ttsResponse = await fetch('/speak', {
                                method: 'POST',
                                headers: { 'Content-Type': 'application/json' },
                                body: JSON.stringify({ text: captionZh })
                            });

                            if (ttsResponse.ok) {
                                const audioBlob = await ttsResponse.blob();
                                const audioUrl = URL.createObjectURL(audioBlob);
                                const audio = new Audio(audioUrl);

                                // Enhanced Mobile UI: Big Custom Button + Native Player
                                const uniqueId = Date.now();
                                const audioHtml = `
                                    <div style="margin-top:15px; padding:15px; background:#f4f4f4; border-radius:12px; border:1px solid #ddd; text-align:center;">
                                        <div style="font-size:12px; color:#666; margin-bottom:10px;">🔊 语音已生成 (CosyVoice2)</div>
                                        
                                        <!-- Big Custom Play/Pause Button -->
                                        <button id="btn-${uniqueId}" onclick="toggleAudio(${uniqueId})" 
                                            style="width:100%; padding:15px; background:white; border:2px solid #4CAF50; color:#4CAF50; border-radius:8px; font-size:16px; font-weight:bold; cursor:pointer; margin-bottom:10px; display:flex; align-items:center; justify-content:center; gap:8px;">
                                            <span>▶️</span> 点击播放语音
                                        </button>

                                        <!-- Native Player (hidden or small, for seeking) -->
                                        <audio id="audio-${uniqueId}" onended="resetAudio(${uniqueId})" controls src="${audioUrl}" style="width: 100%; height: 40px; border-radius:20px;"></audio>
                                    </div>
                                `;
                                resultContent.innerHTML += audioHtml;

                                // Reset button state
                                uploadBtn.innerHTML = `🚀 开始识别`;

                                // Auto-play attempt (with icon update if successful)
                                audio.play()
                                    .then(() => {
                                        // If auto-play succeeds, update the button state immediately
                                        // We need to wait for the DOM to update first
                                        setTimeout(() => {
                                            const btn = document.getElementById('btn-' + uniqueId);
                                            if (btn) {
                                                btn.innerHTML = '<span>⏸️</span> 点击暂停';
                                                btn.style.color = '#ff9800';
                                                btn.style.borderColor = '#ff9800';
                                            }
                                        }, 100);
                                    })
                                    .catch(e => console.warn("Auto-play prevented:", e));

                                // Auto-scroll to show the player
                                requestAnimationFrame(() => {
                                    resultContent.lastElementChild.scrollIntoView({ behavior: "smooth", block: "end" });
                                });
                            }
                        } catch (ttsErr) {
                            console.log("TTS failed: " + ttsErr);
                            uploadBtn.innerHTML = `⚠️ 语音生成失败`;
                        }

                    } else {
                        // 404 implies Lesson 25 backend or error
                        console.log("Translation endpoint not found or error, skipping step 2.");
                    }
                } catch (err) {
                    console.log("Translation step skipped: " + err);
                }

            } catch (error) {
                clearInterval(timerInterval);
                alert('上传失败，请重试');
                console.error(error);
                resultContent.innerText = "❌ 系统错误: " + error.message;
            } finally {
                // UI State: Reset
                uploadBtn.disabled = false;
                uploadBtn.innerText = '🚀 开始识别';
            }
        });
    </script>
</body>

</html>

In [ ]:
%%writefile app_v2.py
import os
# Fix for MPS 'aten::isin' error: Enable fallback to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
# Suppress huggingface/tokenizers parallelism warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import uvicorn
from fastapi import FastAPI, UploadFile, File, Body
from fastapi.responses import HTMLResponse, Response
from pydantic import BaseModel
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import io
import socket
import qrcode
import sys
import torch
import pathlib
import pathlib
import time
import warnings # Added for warning suppression
from openai import OpenAI  # <--- New Import for Lesson 26
from fastapi.responses import StreamingResponse # For Audio Stream

# Filter out specific library warnings to keep console clean for students
warnings.filterwarnings("ignore", message=".*To copy construct from a tensor.*")
warnings.filterwarnings("ignore", category=UserWarning, module="transformers")

# 1. 初始化 FastAPI 应用
app = FastAPI(title="Lesson 26 AI Vision Translator")

# ==========================================
# ☁️ Cloud API Configuration (Lesson 26 New)
# ==========================================
# 尝试从环境变量获取 Key，如果没有则留空 (会报错提示)
# 尝试从环境变量获取 Key
SILICONFLOW_API_KEY = os.getenv("SILICONFLOW_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 检查是否有可用的 Key
VALID_API_KEY = SILICONFLOW_API_KEY or OPENAI_API_KEY

SILICONFLOW_BASE_URL = "https://api.siliconflow.cn/v1"
CLOUD_MODEL_NAME = "deepseek-ai/DeepSeek-V3.2" 

if not VALID_API_KEY:
    print("\n" + "!"*50)
    print("⛔️ 致命错误: 未检测到 API Key！")
    print("--------------------------------------------------")
    print("请至少设置以下其中一个环境变量:")
    print("1. export SILICONFLOW_API_KEY='sk-...' (推荐)")
    print("2. export OPENAI_API_KEY='sk-...' (备选)")
    print("!"*50 + "\n")
    sys.exit(1) # Fail fast: 缺少核心依赖直接退出

# Mock flash_attn for Mac compatibility
from unittest.mock import MagicMock
sys.modules["flash_attn"] = MagicMock()
sys.modules["flash_attn"].__spec__ = MagicMock()

# Helper: Get Local IP
def get_local_ip():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip = s.getsockname()[0]
        s.close()
        return ip
    except:
        return "127.0.0.1"

# 2. 预加载模型 (使用 Microsoft Florence-2-base)
local_model_path = "./models/florence-2-base"
if os.path.exists(local_model_path):
    print(f"📂 发现本地模型 path: {local_model_path}")
    model_id = local_model_path
else:
    print(f"⚠️ 未找到本地模型，准备从 HuggingFace 下载...")
    model_id = "microsoft/Florence-2-base"

device = "mps" if torch.backends.mps.is_available() else "cpu"
torch_dtype = torch.float16 if device != "cpu" else torch.float32

model = None
processor = None

try:
    print("="*50)
    print(f"🚀 正在加载 Florence-2 模型...")
    print(f"📂 模型来源: {model_id}")
    print(f"🖥️  运行设备: {device.upper()}")
    
    local_files_only = False
    if os.path.isdir(model_id):
        print(f"🔌 检测到本地路径，启用离线模式 (local_files_only=True)")
        local_files_only = True
        os.environ["HF_DATASETS_OFFLINE"] = "1"
        os.environ["TRANSFORMERS_OFFLINE"] = "1"
    
    model = AutoModelForCausalLM.from_pretrained(
        model_id, 
        trust_remote_code=True, 
        torch_dtype=torch_dtype,
        local_files_only=local_files_only
    ).to(device)
    
    processor = AutoProcessor.from_pretrained(
        model_id, 
        trust_remote_code=True,
        local_files_only=local_files_only
    )
    
    print("✨ Florence-2 模型加载成功！")
    print("="*50)
except Exception as e:
    print(f"\n❌ 模型加载失败: {e}")

# 3. 模型预热
def warmup_model():
    if not model or not processor: return
    print("🔥 正在预热模型... (消除首次推理卡顿)")
    try:
        dummy_img = Image.new('RGB', (64, 64), color='white')
        dummy_prompt = "<CAPTION>"
        inputs = processor(text=dummy_prompt, images=dummy_img, return_tensors="pt").to(device, torch_dtype)
        model.generate(
            input_ids=inputs["input_ids"],
            pixel_values=inputs["pixel_values"],
            max_new_tokens=5, 
            do_sample=False,
            num_beams=1,
        )
        print("✅ 模型预热完成！")
    except Exception as e:
        print(f"⚠️ 预热失败 (不影响主功能): {e}")

warmup_model()

# ==========================================
# 🧠 new helper: Cloud Translation
# ==========================================
def translate_text(text: str) -> str:
    """调用云端大模型将英文翻译成中文"""
    # 尝试多种方式获取 Key
    api_key = SILICONFLOW_API_KEY or os.getenv("OPENAI_API_KEY")
    
    # 即使 api_key 为 None，我们也尝试初始化，因为 OpenAI SDK 可能有自己的配置加载机制
    try:
        client = OpenAI(api_key=api_key, base_url=SILICONFLOW_BASE_URL)
        response = client.chat.completions.create(
            model=CLOUD_MODEL_NAME,
            messages=[
                {"role": "system", "content": "你是一个专业的翻译助手。请将用户的英文输入直接翻译成中文，不要添加任何解释。"},
                {"role": "user", "content": text},
            ],
            stream=False
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"❌ 翻译出错: {e}")
        return f"翻译失败: {str(e)} (可能是 API Key 未配置)"

        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"❌ 翻译出错: {e}")
        return f"翻译失败: {str(e)} (可能是 API Key 未配置)"

# Pydantic Model for API
class TranslationRequest(BaseModel):
    text: str

class TTSRequest(BaseModel):
    text: str

# ==========================================
# 🚦 Routes
# ==========================================

@app.get("/", response_class=HTMLResponse)
def home():
    """Serve the frontend HTML"""
    return pathlib.Path("lesson26.html").read_text(encoding="utf-8")

@app.get("/qrcode")
def get_qr_image():
    ip = get_local_ip()
    url = f"http://{ip}:8000"
    qr = qrcode.QRCode(box_size=10, border=4)
    qr.add_data(url)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    buf.seek(0)
    return Response(content=buf.getvalue(), media_type="image/png")

@app.post("/upload")
async def upload_image(file: UploadFile = File(...)):
    """
    Step 1: 视觉识别 (Vision Only)
    接收图片 -> Florence-2 生成英文描述 -> 返回英文
    """
    global model, processor
    
    if not model or not processor:
        return {"error": "Model not loaded."}

    try:
        contents = await file.read()
        image = Image.open(io.BytesIO(contents))
        if image.mode != "RGB":
            image = image.convert("RGB")
        
        # 构造提示词 task
        prompt = "<MORE_DETAILED_CAPTION>"
        
        # 打印日志
        print(f"\n📸 收到图片: {file.filename}")
        
        # 计时开始
        start_time = time.time()
        
        # 1. 本地视觉推理 (Florence-2)
        inputs = processor(text=prompt, images=image, return_tensors="pt").to(device, torch_dtype)
        generated_ids = model.generate(
            input_ids=inputs["input_ids"],
            pixel_values=inputs["pixel_values"],
            max_new_tokens=1024,
            do_sample=False,
            num_beams=3,
        )
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
        parsed_answer = processor.post_process_generation(
            generated_text, 
            task=prompt, 
            image_size=(image.width, image.height)
        )
        caption_en = parsed_answer[prompt]
        
        # 计时结束
        end_time = time.time()
        cost_time = round(end_time - start_time, 2)
        
        # 打印日志
        print(f"⏱️ 视觉耗时: {cost_time}s")
        print(f"🤖 EN: {caption_en}")
        
        return {
            "label": caption_en,     # 兼容旧逻辑，前端如果是旧版也会直接显示英文
            "caption_en": caption_en, # 显式字段
            "cost_time": cost_time
        }
    except Exception as e:
        return {"error": str(e)}

@app.post("/translate")
async def translate_endpoint(request: TranslationRequest):
    """
    Step 2: 云端翻译 (Language Only)
    接收英文 -> 调用 API -> 返回中文
    """
    print(f"☁️ 收到翻译请求: {request.text[:50]}...")
    start_time = time.time()
    
    caption_zh = translate_text(request.text)
    
    end_time = time.time()
    cost_time = round(end_time - start_time, 2)
    
    print(f"🇨🇳 ZH: {caption_zh}")
    print(f"⏱️ 翻译耗时: {cost_time}s")
    
    return {
        "caption_zh": caption_zh,
        "cost_time": cost_time
    }

@app.post("/speak")
async def speak_endpoint(request: TTSRequest):
    """
    Step 3: 文本转语音 (TTS)
    接收中文 -> 调用 SiliconFlow API -> 返回 MP3 音频流
    """
    print(f"🔈 收到 TTS 请求: {request.text[:50]}...")
    
    # 尝试多种方式获取 Key
    api_key = SILICONFLOW_API_KEY or OPENAI_API_KEY
    client = OpenAI(api_key=api_key, base_url=SILICONFLOW_BASE_URL)

    def generate_audio():
        with client.audio.speech.with_streaming_response.create(
            model="FunAudioLLM/CosyVoice2-0.5B", 
            voice="FunAudioLLM/CosyVoice2-0.5B:anna", # 音色
            input=request.text, 
            response_format="mp3"
        ) as response:
            for chunk in response.iter_bytes():
                yield chunk

    return StreamingResponse(generate_audio(), media_type="audio/mpeg")

if __name__ == "__main__":
    ip = get_local_ip()
    port = 8000
    url = f"http://{ip}:{port}/docs"
    
    print("\n" + "="*50)
    print(f"🚀 Lesson 26 AI Vision Translator 启动中...")
    print(f"Running on: {url}")
    print("="*50)
    
    # 打印二维码供扫描
    try:
        qr = qrcode.QRCode()
        qr.add_data(url)
        qr.print_ascii(invert=True)
        print(f"\n📱 手机扫码体验: {url}")
    except:
        pass
    
    print("\n按 Ctrl+C 停止服务")
    
    uvicorn.run(app, host="0.0.0.0", port=port)


In [ ]:
!python app_v2.py